<a href="https://colab.research.google.com/github/Sravani-05/Assignment-6-frequent-pattern-mining-algorithms/blob/main/frequent_pattern_mining_algorithms_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install opendatasets
!pip install apyori
!pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5973 sha256=cb8372078e44985c5d235d5cfde26dce7dccb3ee59fad606bb1d555ea109f4c9
  Stored in directory: /root/.cache/pip/wheels/1b/02/6c/a45230be8603bd95c0a51cd2b289aefdd860c1a100eab73661
Successfully built apyori
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
%pip install mlxtend --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 13.5 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [6]:
!gdown 12ixZGdrcwr_c4nVnjVEonFJLiFrbBzsD


Downloading...
From: https://drive.google.com/uc?id=12ixZGdrcwr_c4nVnjVEonFJLiFrbBzsD
To: /content/kaggle.json
100% 65.0/65.0 [00:00<00:00, 105kB/s]


In [8]:
import opendatasets as od
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import association_rules, apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

**frequent pattern mining apriori**

In [27]:
od.download('https://www.kaggle.com/datasets/aslanahmedov/market-basket-analysis?select=Assignment-1_Data.csv')

100%|██████████| 22.8M/22.8M [00:01<00:00, 18.6MB/s]


In [28]:
df = pd.read_csv('/content/the-bread-basket/bread basket.csv')

In [29]:
df.head()

,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,30-10-2016 09:58,morning,weekend
1,2,Scandinavian,30-10-2016 10:05,morning,weekend
2,2,Scandinavian,30-10-2016 10:05,morning,weekend
3,3,Hot chocolate,30-10-2016 10:07,morning,weekend
4,3,Jam,30-10-2016 10:07,morning,weekend


In [30]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20507 entries, 0 to 20506
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Transaction      20507 non-null  int64 
 1   Item             20507 non-null  object
 2   date_time        20507 non-null  object
 3   period_day       20507 non-null  object
 4   weekday_weekend  20507 non-null  object
dtypes: int64(1), object(4)
memory usage: 801.2+ KB


In [31]:
df.shape


(20507, 5)

In [32]:
df.Item.nunique(), df.Transaction.nunique()


(94, 9465)

In [33]:
df_transactions = df.groupby(['Transaction', 'Item'])['Item'].count().reset_index(name='Frequency')
df_transactions

,Transaction,Item,Frequency
0,1,Bread,1
1,2,Scandinavian,2
2,3,Cookies,1
3,3,Hot chocolate,1
4,3,Jam,1
...,...,...,...
18882,9682,Tacos/Fajita,1
18883,9682,Tea,1
18884,9683,Coffee,1
18885,9683,Pastry,1


In [34]:
transactions = df_transactions.pivot_table(index='Transaction', columns='Item', values='Frequency', aggfunc='sum').fillna(0)
transactions.head()

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
def binary_encode(x):
    if x == 0:
        return 0
    if x >= 1:
        return 1

transactions = transactions.applymap(binary_encode)
transactions.head()

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
frequent_sets = apriori(transactions, min_support=0.01, use_colnames=True)

/usr/local/lib/python3.8/dist-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [37]:
# Rules from frequent itemset 
rules = association_rules(frequent_sets, metric='lift', min_threshold=1)

# Sort values by confidence
rules.sort_values('confidence', ascending=False, inplace=True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
31,(Toast),(Coffee),0.033597,0.478394,0.023666,0.704403,1.472431,0.007593,1.764582
29,(Spanish Brunch),(Coffee),0.018172,0.478394,0.010882,0.598837,1.251766,0.002189,1.300235
19,(Medialuna),(Coffee),0.061807,0.478394,0.035182,0.569231,1.189878,0.005614,1.210871
22,(Pastry),(Coffee),0.086107,0.478394,0.047544,0.552147,1.154168,0.006351,1.164682
1,(Alfajores),(Coffee),0.036344,0.478394,0.019651,0.540698,1.130235,0.002264,1.135648
17,(Juice),(Coffee),0.038563,0.478394,0.020602,0.534247,1.116750,0.002154,1.119919
24,(Sandwich),(Coffee),0.071844,0.478394,0.038246,0.532353,1.112792,0.003877,1.115384
7,(Cake),(Coffee),0.103856,0.478394,0.054728,0.526958,1.101515,0.005044,1.102664
27,(Scone),(Coffee),0.034548,0.478394,0.018067,0.522936,1.093107,0.001539,1.093366
13,(Cookies),(Coffee),0.054411,0.478394,0.028209,0.518447,1.083723,0.002179,1.083174


**fpgrowth techniques**

In [38]:
od.download('https://www.kaggle.com/datasets/aslanahmedov/market-basket-analysis?select=Assignment-1_Data.csv')

Skipping, found downloaded files in "./market-basket-analysis" (use force=True to force download)


In [39]:
df = pd.read_csv('/content/the-bread-basket/bread basket.csv')

In [40]:
df.head()

,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,30-10-2016 09:58,morning,weekend
1,2,Scandinavian,30-10-2016 10:05,morning,weekend
2,2,Scandinavian,30-10-2016 10:05,morning,weekend
3,3,Hot chocolate,30-10-2016 10:07,morning,weekend
4,3,Jam,30-10-2016 10:07,morning,weekend


In [41]:
df.shape


(20507, 5)

In [43]:
rows = df.shape[0]
cols = df.shape[1]

transaction = []
for i in range(rows):
    transaction.append([str(df.values[i,j]) for j in range(cols)])

transaction = np.array(transaction)

# Replace values with True/False. True if item is present in transaction and False otherwise
encoder = TransactionEncoder()
df_transformed = encoder.fit(transaction).transform(transaction)
df = pd.DataFrame(df_transformed, columns=encoder.columns_)

df.head()

,01-01-2017 01:21,01-02-2017 08:37,01-02-2017 08:40,01-02-2017 09:05,01-02-2017 09:07,01-02-2017 09:31,01-02-2017 09:37,01-02-2017 10:00,01-02-2017 10:28,01-02-2017 10:32,...,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge,afternoon,evening,morning,night,weekday,weekend
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True


In [45]:
from mlxtend.frequent_patterns import fpgrowth

In [46]:
item_sets = fpgrowth(df, min_support=0.1, use_colnames=True)
item_sets.head(10)

,support,itemsets
0,0.409811,(morning)
1,0.375482,(weekend)
2,0.162140,(Bread)
3,0.266787,(Coffee)
4,0.564149,(afternoon)
5,0.624518,(weekday)
6,0.252304,"(morning, weekday)"
7,0.157507,"(weekend, morning)"
8,0.209489,"(weekend, afternoon)"
9,0.102014,"(Bread, weekday)"


In [47]:
# Create association rules for frequent items
rules = association_rules(item_sets, metric='lift', min_threshold=1)

# Sort values by confidence
rules.sort_values('confidence', ascending=False, inplace=True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(Coffee),(weekday),0.266787,0.624518,0.172770,0.647596,1.036953,0.006157,1.065487
2,(Bread),(weekday),0.162140,0.624518,0.102014,0.629173,1.007453,0.000755,1.012552
8,(afternoon),(weekday),0.564149,0.624518,0.354659,0.628663,1.006636,0.002338,1.011161
9,(weekday),(afternoon),0.624518,0.564149,0.354659,0.567893,1.006636,0.002338,1.008664
4,(Coffee),(morning),0.266787,0.409811,0.124884,0.468105,1.142244,0.015552,1.109595
0,(weekend),(morning),0.375482,0.409811,0.157507,0.419481,1.023594,0.003631,1.016656
1,(morning),(weekend),0.409811,0.375482,0.157507,0.384341,1.023594,0.003631,1.014390
5,(morning),(Coffee),0.409811,0.266787,0.124884,0.304736,1.142244,0.015552,1.054582
7,(weekday),(Coffee),0.624518,0.266787,0.172770,0.276646,1.036953,0.006157,1.013629
3,(weekday),(Bread),0.624518,0.162140,0.102014,0.163348,1.007453,0.000755,1.001444


In [48]:
# Create association rules for frequent items
rules = association_rules(item_sets, metric='lift', min_threshold=1)

# Sort values by confidence
rules.sort_values('confidence', ascending=False, inplace=True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(Coffee),(weekday),0.266787,0.624518,0.172770,0.647596,1.036953,0.006157,1.065487
2,(Bread),(weekday),0.162140,0.624518,0.102014,0.629173,1.007453,0.000755,1.012552
8,(afternoon),(weekday),0.564149,0.624518,0.354659,0.628663,1.006636,0.002338,1.011161
9,(weekday),(afternoon),0.624518,0.564149,0.354659,0.567893,1.006636,0.002338,1.008664
4,(Coffee),(morning),0.266787,0.409811,0.124884,0.468105,1.142244,0.015552,1.109595
0,(weekend),(morning),0.375482,0.409811,0.157507,0.419481,1.023594,0.003631,1.016656
1,(morning),(weekend),0.409811,0.375482,0.157507,0.384341,1.023594,0.003631,1.014390
5,(morning),(Coffee),0.409811,0.266787,0.124884,0.304736,1.142244,0.015552,1.054582
7,(weekday),(Coffee),0.624518,0.266787,0.172770,0.276646,1.036953,0.006157,1.013629
3,(weekday),(Bread),0.624518,0.162140,0.102014,0.163348,1.007453,0.000755,1.001444
